In [1]:
import pandas as pd
import logging

from src import eda, datasets

In [2]:
logging.getLogger().setLevel(logging.DEBUG)

In [3]:
project_pack = datasets.DatasetPack(restore=True)

INFO:root:Restoring backup: /Users/bubblegum_doubledrops/Library/Mobile Documents/com~apple~CloudDocs/0prio - Important heavy backups/IronHack/big_projects/end-to-end-data-analysis/datasets/_dataset_pack_end-to-end-data-analysis.json
INFO:root:Restoring dataframe from backup: /Users/bubblegum_doubledrops/Library/Mobile Documents/com~apple~CloudDocs/0prio - Important heavy backups/IronHack/big_projects/end-to-end-data-analysis/datasets/df_bandcamp_sales.pkl
INFO:root:Restoring dataframe from backup: /Users/bubblegum_doubledrops/Library/Mobile Documents/com~apple~CloudDocs/0prio - Important heavy backups/IronHack/big_projects/end-to-end-data-analysis/datasets/df_discogs_artists.pkl
INFO:root:Restoring dataframe from backup: /Users/bubblegum_doubledrops/Library/Mobile Documents/com~apple~CloudDocs/0prio - Important heavy backups/IronHack/big_projects/end-to-end-data-analysis/datasets/df_discogs_releases.pkl
INFO:root:Backup restored successfully from /Users/bubblegum_doubledrops/Library/M

In [4]:
auto_eda_pack = {label: eda.Auto_EDA(dataset) for label, dataset in project_pack.dictionary.items()}

In [5]:
print("Copy the output as code to run in the next cell")
# TODO: implement DatasetPack.labels() to return a list
for label in project_pack.dictionary.keys():
    print(f"{label} = project_pack.dictionary['{label}'].dataframe")

Copy the output as code to run in the next cell
df_bandcamp_sales = project_pack.dictionary['df_bandcamp_sales'].dataframe
df_discogs_artists = project_pack.dictionary['df_discogs_artists'].dataframe
df_discogs_releases = project_pack.dictionary['df_discogs_releases'].dataframe


In [6]:
df_bandcamp_sales = project_pack.dictionary['df_bandcamp_sales'].dataframe
df_discogs_artists = project_pack.dictionary['df_discogs_artists'].dataframe
df_discogs_releases = project_pack.dictionary['df_discogs_releases'].dataframe

### Frequency counts for df_bandcamp_sales product types

As we see from the auto-EDA output:

- Column "item_type" has 4 unique values.

  -- Unique values are:
 ['a' 'p' 't' 'b']

- Column "slug_type" has 3 unique values.

  -- Unique values are:
 ['a' 't' 'p' nan]

Both columns contain product type codes.

```
-- a: for albums; 
--- in 'item_type': a stands for all album types;
--- in 'slug_type': a stands only for digital albums;
-- t: for digital single tracks;
-- p: for physical items: merchendise or hard media (vynil/CD releases);
--- in 'item_type': p stands for all physical items;
--- in 'slug_type': p stands only for merch;
-- b: type-surprise, it was missing from the source description.
```

### Frequency counts for df_discogs_artists

As we see from the auto-EDA output:
- Column "artist_data_quality" has 6 unique values.

  -- Unique values are:

 ['Needs Vote' 'Correct' 'Needs Major Changes' 'Complete and Correct'
 'Needs Minor Changes' 'Entirely Incorrect']

### Frequency counts for df_discogs_releases

As we see from the auto-EDA output:

- Column "genre" has 15 unique values.

  -- Unique values are:

 ['Electronic' 'Hip Hop' 'Non-Music' 'Jazz' 'Rock' 'Latin' 'Funk / Soul'
 'Stage & Screen' 'Pop' 'Reggae' 'Folk, World, & Country' 'Classical'
 'Blues' 'Brass & Military' "Children's" nan]

- Column "format" has 57 unique values.

## Backup

In [ ]:
for _, dataset in project_pack.dictionary.items():
    dataset.backup()

project_pack.backup_pack()